In [1]:
from typing_extensions import TypedDict, Annotated, List, Literal
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage, SystemMessage
from langgraph.graph import add_messages
from typing import Dict, List, Optional, Union, Literal
from pydantic import BaseModel, Field
from IPython.display import Image, display 
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

In [2]:
## documents
class DocumentSection(BaseModel):
    title: str = Field(description="The title of the section")
    content: str = Field(description="The content of the section")
    

## security review
class SecurityReview(TypedDict):
    sec_id : str
    review : str
    file_path : str
    recommendation : str
    priority : Literal["high", "medium", "low"]
    
## test cases
class TestCase(TypedDict):
    test_id : str 
    description : str 
    steps : List[str]
    status : Literal["draft", "pass", "fail"] = "draft"

In [3]:
## Project Requirementsz
class ProjectRequirements(BaseModel):
    title: str = Field(description="The title of the project")
    description: str = Field(description="The description of the project")
    requirements : List[str] = Field(description="The requirements of the project")

In [4]:
## User story
class UserStory(BaseModel):
    story_id : str = Field(description="The id of the user story")
    title : str = Field(description="The title of the user story")
    description : str = Field(description="The description of the user story")
    acceptance_criteria : List[str] = Field(description="The acceptance criteria of the user story")

In [5]:
class SDLCState(BaseModel):
    project_requirements : ProjectRequirements
    revised_count : int = Field(default=0, description="The number of times the revised")
    # User story
    user_stories : List[UserStory] = []
    user_story_messages : Annotated[list, add_messages] = []
    user_story_status : Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"
    
    document_type: Literal["functional", "technical"] = "functional"
    # functional documents
    functional_documents : List[DocumentSection] = []
    functional_messages : Annotated[list, add_messages] = []
    functional_status: Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"
    # technical documents
    technical_documents : List[DocumentSection] = []
    technical_messages : Annotated[list, add_messages] = []
    technical_status: Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"
    
    code_type : Literal["frontend", "backend"] = "frontend"
    # frontend code
    frontend_code : str = Field(default='', description="The frontend code")
    frontend_messages: Annotated[list, add_messages] = []
    frontend_status: Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"
    # backend code
    backend_code : str = Field(default= '', description="The backend code")
    backend_messages: Annotated[list, add_messages] = []
    backend_status: Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"

    # security review
    security_reviews : List[SecurityReview] = []
    security_reviews_messages: Annotated[list, add_messages] = []
    security_reviews_status : Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"

    ## test cases
    test_cases : List[TestCase] = []
    test_cases_messages: Annotated[list, add_messages] = []
    test_cases_status : Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"

## Helper functions

In [6]:
def generate_user_stories_from_llm(project_title, project_description, requirements):
    user_stories = [{'story_id': 'US-001',
        'title': 'Secure Account with Multi-Factor Authentication',
        'description': 'As a user, I want to secure my account with multi-factor authentication so that my transactions are protected.',
        'acceptance_criteria': ['User can enable biometric authentication (fingerprint and facial recognition).',
        'User can set up MPIN for additional security.',
        'User can enable two-factor authentication for transactions.',
        'System alerts user of authentication failures.']}]
    
    return user_stories


def revised_user_stories_from_llm(user_stories, user_review):
    revised_user_stories = [{'story_id': 'US-001',
        'title': 'Secure Account with Multi-Factor Authentication',
        'description': 'As a user, I want to secure my account with multi-factor authentication so that my transactions are protected.',
        'acceptance_criteria': ['User can enable biometric authentication (fingerprint and facial recognition).',
        'User can set up MPIN for additional security.',
        'User can enable two-factor authentication for transactions.',
        'System alerts user of authentication failures.']},
        {'story_id': 'US-002',
        'title': 'Link Multiple Bank Accounts',
        'description': 'As a user, I want to link multiple bank accounts so that I can transfer funds instantly.',
        'acceptance_criteria': ['User can add multiple bank accounts to their profile.',
        'User can select any linked account for fund transfers.',
        'Linked accounts display current balance.',
        'System confirms successful linking of accounts.']}]
    return revised_user_stories

def generate_design_documents_from_llm(doc_type, user_stories, project_requirements, user_feedback = ""):
    documents = []
    if doc_type == "functional":
        documents = generate_functional_design_documents_from_llm(user_stories, project_requirements, user_feedback)
    elif doc_type == "technical":
        documents = generate_technical_design_documents_from_llm(user_stories, project_requirements, user_feedback)
    return documents

def generate_functional_design_documents_from_llm(user_stories, project_requirements, user_feedback = ""):
    functional_documents = []
    if user_feedback == "":
        document = DocumentSection(
        title = f"FUNCTIONAL REQUIREMENTS",
        content = """
        FR-1: The system shall provide a "Forgot Password" option on the login page.  
        FR-2: The system shall send a time-limited reset link to the user's registered email address.  
        FR-3: The system shall validate new passwords against password policy rules.  
        FR-4: The system shall notify users of successful password reset and redirect them to the login page.
        """)
        functional_documents = [document]
    else: 
        document = DocumentSection(
            title = f"Revised FUNCTIONAL REQUIREMENTS",
            content = """
            FR-1: The system shall provide a "Forgot Password" option on the login page.  
            FR-2: The system shall send a time-limited reset link to the user's registered email address.  
            FR-3: The system shall validate new passwords against password policy rules.  
            FR-4: The system shall notify users of successful password reset and redirect them to the login page.
        """)
        functional_documents = [document]
    return functional_documents


def generate_technical_design_documents_from_llm(user_stories, project_requirements, user_feedback = ""):
    technical_documents = []
    if user_feedback == "":
        document = DocumentSection(
        title =  "Technical Document: MODULES & COMPONENTS DESIGN",
        content = """
        - **Auth API Module:** Exposes RESTful endpoints for password reset request and confirmation  
        - **Token Service Module:** Generates, hashes, stores, and validates password reset tokens  
        - **Notification Service:** Sends password reset links securely via AWS SES  
        - **Audit Logging Module:** Captures all user-initiated reset activities for security auditing
        """),
        technical_documents = [document]
    else: 
        document = DocumentSection(
            title = "Revised Technical Document: MODULES & COMPONENTS DESIGN",
        content = """
        - **Auth API Module:** Exposes RESTful endpoints for password reset request and confirmation  
        - **Token Service Module:** Generates, hashes, stores, and validates password reset tokens  
        - **Notification Service:** Sends password reset links securely via AWS SES  
        - **Audit Logging Module:** Captures all user-initiated reset activities for security auditing
        """),
        technical_documents = [document]
    return technical_documents

## Node functions

### User stories

In [7]:
## User stories 
def process_project_requirements(state : SDLCState) -> SDLCState:
    print("In process_project_requirements...")
    return {
        "user_story_messages" : AIMessage(content="I've received your project requirements. I'll now generate user stories based on these requirements.")
    }
    
def generate_user_stories(state : SDLCState) -> SDLCState:
    print("In generate_user_stories...") 
    project_title = state.project_requirements.title
    project_description = state.project_requirements.description
    requirements = state.project_requirements.requirements
    user_stories = generate_user_stories_from_llm(project_title, project_description, requirements);
    
    return {
        "user_story_status" : 'pending_approval',
        "user_stories" : user_stories,
        "user_story_messages" : AIMessage(
            content = f"Based on your requirements, I've generated {len(user_stories)} user stories. Please review these user stories and provide feedback or type 'Approved' if you're satisfied."
        ),
        "revised_count" : 0
    }

def review_user_stories(state : SDLCState) -> SDLCState:
    print("In review_user_stories...")
    user_review = state.user_story_messages[-1].content
    user_review = user_review.lower().strip()
    print("user_review : ", user_review)  
    if user_review == "approved":
        return {
            "user_story_messages" : AIMessage(content="Great! Your user stories have been finalized. You can now proceed with your design process."),
            "user_story_status" : "approved",
        }
    else:
        return {
            "user_story_messages" :  AIMessage(content="I've received your feedback. I'll revise the user stories accordingly."),
            "user_story_status" : "feedback"
        }

def should_revise_user_stories(state : SDLCState) -> Literal["feedback", "approved"]:
    return "approved" if state.user_story_status == "approved" else "feedback"   

def revised_user_stories(state : SDLCState) -> SDLCState:
    print("In revised_user_stories...")
    user_review = state.user_story_messages[-2].content
    user_review = user_review.lower().strip()
    if state.user_story_status == 'feedback':
        revised_count = state.revised_count + 1
        print("revised_count : ", revised_count)
        if revised_count == 3:
            return {
                "user_story_messages" :  AIMessage(
                    content = f"User stories have been revision maxed out. Please review the user stories and continue with the next step."
                ),
                "user_story_status" : "approved"
            }
        user_stories = revised_user_stories_from_llm(state.user_stories, user_review)
        return {
                "user_stories" : user_stories,
                "user_story_messages" : AIMessage(
                    content = "I've revised the user stories based on your feedback.\n\nPlease review these updated user stories and provide additional feedback or type 'Approved' if you're satisfied."),
                "user_story_status" : "pending_approval",
                "revised_count" : revised_count
            }

### functional documents

In [8]:
def create_functional_documents(state : SDLCState) -> SDLCState:
    print("In create_functional_documents...")
    doc_type = "functional"
    user_stories = state.user_stories
    project_requirements = state.project_requirements
    documents = generate_design_documents_from_llm(doc_type, user_stories, project_requirements)

    return {
        f"{doc_type}_documents": documents,
        f"{doc_type}_status": 'pending_approval',
        f"{doc_type}_messages": AIMessage(
            content=f"Please review {doc_type} design document and provide feedback or type 'Approved' if you're satisfied."
        ),
    }

def review_functional_documents(state : SDLCState) -> SDLCState:
    print("In review_functional_documents...")
    doc_type = "functional"
    user_feedback = state.functional_messages[-1].content.lower().strip()
    print("user feedback:", user_feedback)
    approved = (user_feedback == "approved")
    response = {
        f"{doc_type}_messages": AIMessage(
            content="Great! Your design documents have been finalized. You can now proceed with your design process."
            if approved else
            "I've received your feedback. I'll revise the design documents accordingly."
        ),
        f"{doc_type}_status": "approved" if approved else "feedback",
    }
    return response

def should_revise_design_documents(state : SDLCState) -> Literal["feedback", "approved"]:
    if state.document_type == "functional":
        return "approved" if state.functional_status == "approved" else "feedback"   
    else:
        return "approved" if state.technical_status == "approved" else "feedback"   

def revise_functional_documents(state : SDLCState) -> SDLCState:
    print("In revise_functional_documents...")
    doc_type = "functional"
    user_feedback = state.functional_messages[-2].content.lower().strip()
    print("user feedback:", user_feedback)
    revised_count = state.revised_count + 1
    print("revised_count :", revised_count)

    if revised_count == 3:
        return {
            f"{doc_type}_messages": AIMessage(
                content="Design documents have been revision maxed out. Please review the these documents and continue with the next step."
            ),
            f"{doc_type}_status": "approved",
        }

    documents = generate_design_documents_from_llm(doc_type, state.functional_documents, state.technical_documents, user_feedback)

    return {
        f"{doc_type}_documents": documents,
        f"{doc_type}_messages": AIMessage(
            content=f"Please review revised {doc_type} design documents and provide additional feedback or type 'Approved' if you're satisfied."
        ),
        f"{doc_type}_status": "pending_approval",
        "revised_count": revised_count
    }


### technical documents

In [9]:
def create_technical_documents(state : SDLCState) -> SDLCState:
    print("In create_technical_documents...")
    doc_type = "technical"
    user_stories = state.user_stories
    project_requirements = state.project_requirements
    documents = generate_design_documents_from_llm(doc_type, user_stories, project_requirements)

    return {
        "document_type" : doc_type,
        f"{doc_type}_documents": documents,
        f"{doc_type}_status": 'pending_approval',
        f"{doc_type}_messages": AIMessage(
            content=f"Please review {doc_type} design document and provide feedback or type 'Approved' if you're satisfied."
        ),
    }

def review_technical_documents(state : SDLCState) -> SDLCState:
    print("In review_technical_documents...")
    doc_type = "technical"
    user_feedback = state.technical_messages[-1].content.lower().strip()
    print("user feedback:", user_feedback)
    approved = (user_feedback == "approved")
    response = {
        f"{doc_type}_messages": AIMessage(
            content="Great! Your design documents have been finalized. You can now proceed with your design process."
            if approved else
            "I've received your feedback. I'll revise the design documents accordingly."
        ),
        f"{doc_type}_status": "approved" if approved else "feedback",
    }
    return response

def revise_technical_documents(state : SDLCState) -> SDLCState:
    print("In revise_technical_documents...")
    doc_type = "technical"
    user_feedback = state.technical_messages[-2].content.lower().strip()
    print("user feedback:", user_feedback)
    revised_count = state.revised_count + 1
    print("revised_count :", revised_count)

    if revised_count == 3:
        return {
            f"{doc_type}_messages": AIMessage(
                content="Design documents have been revision maxed out. Please review the these documents and continue with the next step."
            ),
            f"{doc_type}_status": "approved",
        }

    documents = generate_design_documents_from_llm(doc_type, state.functional_documents, state.technical_documents, user_feedback)

    return {
        f"{doc_type}_documents": documents,
        f"{doc_type}_messages": AIMessage(
            content=f"Please review revised {doc_type} design documents and provide additional feedback or type 'Approved' if you're satisfied."
        ),
        f"{doc_type}_status": "pending_approval",
        "revised_count": revised_count
    }

## Frontend code

In [23]:
def generate_frontend_code(state : SDLCState) -> SDLCState:
    pass 

def review_frontend_code(state : SDLCState) -> SDLCState:
    pass 

def should_fix_frontend_code(state : SDLCState) -> Literal["feedback", "approved"]:
    return "approved" if state['status'] == 'approved' else 'feedback'

def fix_frontend_code(state : SDLCState) -> SDLCState:
    pass

## Backend Code

In [24]:
def generate_backend_code(state : SDLCState) -> SDLCState:
    pass 

def review_backend_code(state : SDLCState) -> SDLCState:
    pass 

def should_fix_backend_code(state : SDLCState) -> Literal["feedback", "approved"]:
    return "approved" if state['status'] == 'approved' else 'feedback'

def fix_backend_code(state : SDLCState) -> SDLCState:
    pass

## SDLC Workflow

In [25]:

## Node definition
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

sdlc_graph = StateGraph(SDLCState)

# Adding Nodes 
sdlc_graph.add_node("process_project_requirements", process_project_requirements)
sdlc_graph.add_node("generate_user_stories", generate_user_stories)
sdlc_graph.add_node("review_user_stories", review_user_stories)
sdlc_graph.add_node("revised_user_stories", revised_user_stories)

## Functional documents 
sdlc_graph.add_node("create_functional_documents", create_functional_documents)
sdlc_graph.add_node("review_functional_documents", review_functional_documents)
sdlc_graph.add_node("revise_functional_documents", revise_functional_documents)

## Technical documents 
sdlc_graph.add_node("create_technical_documents", create_technical_documents)
sdlc_graph.add_node("review_technical_documents", review_technical_documents)
sdlc_graph.add_node("revise_technical_documents", revise_technical_documents)


## Frontend code 
sdlc_graph.add_node("generate_frontend_code", generate_frontend_code)
sdlc_graph.add_node("review_frontend_code", review_frontend_code)
sdlc_graph.add_node("fix_frontend_code", fix_frontend_code)

## Backend code
sdlc_graph.add_node("generate_backend_code", generate_backend_code)
sdlc_graph.add_node("review_backend_code", review_backend_code)
sdlc_graph.add_node("fix_backend_code", fix_backend_code)

## User stories 
sdlc_graph.add_edge(START, "process_project_requirements")
sdlc_graph.add_edge("process_project_requirements", "generate_user_stories")
sdlc_graph.add_edge("generate_user_stories", "review_user_stories")
sdlc_graph.add_conditional_edges(
    "review_user_stories",
    should_revise_user_stories,
    {
        "feedback": "revised_user_stories",
        "approved": "create_functional_documents"
    }
)
sdlc_graph.add_edge("revised_user_stories", "review_user_stories")

## Design documents 
sdlc_graph.add_edge("create_functional_documents", "review_functional_documents")
sdlc_graph.add_conditional_edges(
    "review_functional_documents",
    should_revise_design_documents,
    {
        "feedback" : "revise_functional_documents",
        "approved" : "create_technical_documents"
    }
)
sdlc_graph.add_edge("revise_functional_documents", "review_functional_documents")
sdlc_graph.add_edge("create_technical_documents", "review_technical_documents")
sdlc_graph.add_conditional_edges(
    "review_technical_documents",
    should_revise_design_documents,
    {
        "feedback" : "revise_technical_documents",
        "approved" : "generate_frontend_code"
    }
)
sdlc_graph.add_edge("revise_technical_documents", "review_technical_documents")

## Frontend code 
sdlc_graph.add_edge("generate_frontend_code", "review_frontend_code")
sdlc_graph.add_conditional_edges(
    "review_frontend_code",
    should_revise_design_documents,
    {
        "feedback" : "fix_frontend_code",
        "approved" : "generate_backend_code"
    }
)
sdlc_graph.add_edge("fix_frontend_code", "review_frontend_code")

## Backend code 
sdlc_graph.add_edge("generate_backend_code", "review_backend_code")
sdlc_graph.add_conditional_edges(
    "review_backend_code",
    should_revise_design_documents,
    {
        "feedback" : "fix_backend_code",
        "approved" : END
    }
)
sdlc_graph.add_edge("fix_frontend_code", "review_backend_code")


# Compile the graph
sdlc_workflow = sdlc_graph.compile(checkpointer=memory, interrupt_before=["review_user_stories", "review_functional_documents", "review_technical_documents", "review_frontend_code"])

In [26]:
from IPython.display import Image, display
display(Image(sdlc_workflow.get_graph().draw_mermaid_png()))

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

# User stories

In [12]:
project_title = "PayMate: Your Ultimate Payment Companion"
project_description = "PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs."
requirements = ["Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.​", "Enable users to link multiple bank accounts and perform instant fund transfers using UPI.", "Provide users with access to instant micro-loans with minimal documentation.", "Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app."]
requirements_string = "\n".join(requirements)

In [13]:
project_requirements = ProjectRequirements(
    title = project_title,
    description = project_description,
    requirements = requirements
)

initial_story_state = {
    "project_requirements" : project_requirements,
    "user_stories": [],
    "user_story_messages": HumanMessage(content=f"{project_requirements}"),
    "user_story_status": "in_progress",
    "revised_count" : 0,
    "document_type" : "functional",
    "code_type" : "frontend"
}

# Thread
thread = {"configurable": {"thread_id": "59511432q9"}}

state = None
for event in sdlc_workflow.stream(initial_story_state, thread, stream_mode="values"):
    state = event

state

In process_project_requirements...
In generate_user_stories...


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 0,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

In [14]:
current_state = sdlc_workflow.get_state(thread) 
current_state.next

('review_user_stories',)

In [15]:
user_feedback = "Add user story to buy insurance from the app"
# sdlc_workflow.update_state(thread, { "user_story_messages" : HumanMessage(content=f'{user_feedback}')})
sdlc_workflow.update_state(thread, { "user_story_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event

state

In review_user_stories...
user_review :  approved
In create_functional_documents...


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 0,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

# Documentation

In [16]:
current_state = sdlc_workflow.get_state(thread) 
current_state.next

('review_functional_documents',)

In [17]:
user_feedback = "Revised the document for buy insurance from the app"
# sdlc_workflow.update_state(thread, { "functional_messages" : HumanMessage(content=f'{user_feedback}')})
sdlc_workflow.update_state(thread, { "functional_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event

state

In review_functional_documents...
user feedback: approved
In create_technical_documents...


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 0,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

In [18]:
## technical document
current_state = sdlc_workflow.get_state(thread) 
current_state.next

('review_technical_documents',)

In [21]:
user_feedback = "Revised the technical document for buy insurance from the app"
# sdlc_workflow.update_state(thread, { "technical_messages" : HumanMessage(content=f'{user_feedback}')})
sdlc_workflow.update_state(thread, { "technical_messages" : HumanMessage(content='Approved')})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event
    
state


In review_technical_documents...
user feedback: approved


/Users/ideyalabs/Desktop/new/sdlc-copilot/venv/lib/python3.12/site-packages/pydantic/main.py:453: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `DocumentSection` - serialized value may not be as expected [input_value=[DocumentSection(title='R...ty auditing\n        ')], input_type=list])
  return self.__pydantic_serializer__.to_python(
/Users/ideyalabs/Desktop/new/sdlc-copilot/venv/lib/python3.12/site-packages/pydantic/main.py:453: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `DocumentSection` - serialized value may not be as expected [input_value=[DocumentSection(title='R...ty auditing\n        ')], input_type=list])
  return self.__pydantic_serializer__.to_python(


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 1,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

In [22]:
## technical document
current_state = sdlc_workflow.get_state(thread) 
current_state.next

()